In [1]:
import pickle
import numpy
from PIL import Image
import IPython.display

import tracking.Tester
import tracking.RecurrentTracker
import tracking.TheanoGruRnn
import tracking.VideoSequence as vs
import tracking.VisualAttention

/home/fmpaezri/.local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
cd ../tracking

/home/fmpaezri/repos/localization-agent/tracking


In [3]:
with open('/home/fmpaezri/repos/localization-agent/model.pkl0', 'r') as modelFile:
    %time rnn = pickle.load(modelFile)

CPU times: user 9.08 s, sys: 332 ms, total: 9.41 s
Wall time: 9.65 s


In [4]:
tracker = tracking.RecurrentTracker.RecurrentTracker(None, rnn)

In [5]:
tester = tracking.Tester.Tester(tracker)

In [6]:
data, labels = tester.loadImageSequence('/home/fmpaezri/datasets/vot-challenge/sequences2014/ball/', newShape=(rnn.imgSize,rnn.imgSize))

In [7]:
data = data[numpy.newaxis, ...].astype(numpy.float32)

In [8]:
labels = labels[numpy.newaxis,...]

In [9]:
print data.shape, labels.shape

(1, 603, 100, 100, 3) (1, 603, 4)


In [10]:
%time iou, predLabels = tester.test(data, labels, None, rnn.batchSize, rnn.imgSize, False, False, labels.shape[1], 4, '/tmp')

CPU times: user 29.8 s, sys: 1.28 s, total: 31 s
Wall time: 16.6 s


In [11]:
iou.mean()

0.050210864310005605

In [12]:
seq = vs.VideoSequence((Image.fromarray(frame.astype(numpy.uint8)) for frame in data[0]))

In [13]:
seq.addBoxes(labels[0], 'green')
seq.addBoxes((predLabels[0]), 'red')

In [14]:
outputName = '/tmp/testSequence1.mp4'
seq.exportToVideo(30, outputName)

In [15]:
IPython.display.HTML(vs.displayHTML(outputName))